# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,18.61,88,87,11.52,SH,1738530566
1,1,papatowai,-46.5619,169.4708,17.40,45,41,5.39,NZ,1738530567
2,2,malango,-2.9627,119.9001,18.70,87,98,1.78,ID,1738530568
3,3,puerto natales,-51.7236,-72.4875,10.25,62,75,10.80,CL,1738530569
4,4,helena valley southeast,46.6153,-111.9216,-3.57,79,100,5.66,US,1738530570


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.95,
    color = "City")

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.copy()
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()
# I only want weather over 0 C. Not a large ask, eh?
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Max Temp"] > 0 , :]
# I am very much a goldilocks person who likes not too humid / cloudy. Moderate conditions, please.
ideal_weather_df = ideal_weather_df.loc[ (ideal_weather_df["Humidity"] > 25) & (ideal_weather_df["Humidity"] < 75) , :]
ideal_weather_df = ideal_weather_df.loc[ (ideal_weather_df["Cloudiness"] > 15) & (ideal_weather_df["Cloudiness"] < 35) , :]

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
62,62,invercargill,-46.4000,168.3500,16.12,73,24,3.58,NZ,1738530638
63,63,pacasmayo,-7.4006,-79.5714,24.82,66,27,5.79,PE,1738530639
95,95,arraial do cabo,-22.9661,-42.0278,26.92,69,20,6.69,BR,1738530412
119,119,kralendijk,12.1500,-68.2667,28.99,65,20,6.69,BQ,1738530705
152,152,saipan,15.1355,145.7010,27.39,74,20,3.60,MP,1738530621
170,170,villazon,-22.0866,-65.5942,19.64,30,29,6.23,BO,1738530767
190,190,tandalti,13.0167,31.8667,23.07,33,21,4.49,SD,1738530790
239,239,chauk,20.8833,94.8167,20.70,48,31,2.57,MM,1738530848
253,253,lake jackson,29.0339,-95.4344,24.02,53,20,2.57,US,1738530810
272,272,san luis de la loma,17.2706,-100.8939,29.49,51,27,3.32,MX,1738530890


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()
hotel_df = hotel_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
62,invercargill,NZ,-46.4000,168.3500,73,
63,pacasmayo,PE,-7.4006,-79.5714,66,
95,arraial do cabo,BR,-22.9661,-42.0278,69,
119,kralendijk,BQ,12.1500,-68.2667,65,
152,saipan,MP,15.1355,145.7010,74,
170,villazon,BO,-22.0866,-65.5942,30,
190,tandalti,SD,13.0167,31.8667,33,
239,chauk,MM,20.8833,94.8167,48,
253,lake jackson,US,29.0339,-95.4344,53,
272,san luis de la loma,MX,17.2706,-100.8939,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories" : "accommodation.hotel",
    "limit" : 1,
    "apiKey" : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index , "Lat"]
    lon = hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url , params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
invercargill - nearest hotel: The Grand
pacasmayo - nearest hotel: Playa Linda
arraial do cabo - nearest hotel: No hotel found
kralendijk - nearest hotel: Hotel Bonaire Inn
saipan - nearest hotel: Chalan Kanoa Beach Hotel
villazon - nearest hotel: Hotel Ideal
tandalti - nearest hotel: No hotel found
chauk - nearest hotel: Royal Chauk
lake jackson - nearest hotel: Candlewood Suites
san luis de la loma - nearest hotel: No hotel found
puerto ayacucho - nearest hotel: Hotel VENETUR Amazonas
new norfolk - nearest hotel: Woodbridge on the Derwent
fortaleza - nearest hotel: Hotel Catedral
meads - nearest hotel: Fairfield Inn & Suites
maryville - nearest hotel: Executive Lodge
bam - nearest hotel: هتل جهانگردی بم
santa ana - nearest hotel: Holiday Inn Express
esquel - nearest hotel: El Quincho
bubaque - nearest hotel: Chez Julio
mugia - nearest hotel: Solpor do Coído
nova sintra - nearest hotel: Residência Ka Dencho
zuera - nearest hotel: Hotel Restaurante Las Galias
geor

,City,Country,Lat,Lng,Humidity,Hotel Name
62,invercargill,NZ,-46.4000,168.3500,73,The Grand
63,pacasmayo,PE,-7.4006,-79.5714,66,Playa Linda
95,arraial do cabo,BR,-22.9661,-42.0278,69,No hotel found
119,kralendijk,BQ,12.1500,-68.2667,65,Hotel Bonaire Inn
152,saipan,MP,15.1355,145.7010,74,Chalan Kanoa Beach Hotel
170,villazon,BO,-22.0866,-65.5942,30,Hotel Ideal
190,tandalti,SD,13.0167,31.8667,33,No hotel found
239,chauk,MM,20.8833,94.8167,48,Royal Chauk
253,lake jackson,US,29.0339,-95.4344,53,Candlewood Suites
272,san luis de la loma,MX,17.2706,-100.8939,51,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.95,
    color = "City",
    hover_cols  = ["Country","Hotel Name"])

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)